<a href="https://colab.research.google.com/github/zahra-teb/Graph-ML-Final-Project/blob/main/Graph_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

We are going to implement some graph regressors(GNNs) for regression task on molecular dataset FreeSolv using Deep Graph Library.

First, let's install required packages.

In [1]:
!pip install dgl -f https://data.dgl.ai/wheels/repo.html

Looking in links: https://data.dgl.ai/wheels/repo.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 52.5 MB/s eta 0:00:00


In [2]:
!pip install dglgo -f https://data.dgl.ai/wheels-test/repo.html

Looking in links: https://data.dgl.ai/wheels-test/repo.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.5/63.5 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.4/52.4 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 74.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.6/485.6 kB 26.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 572.7/572.7 kB 43.5 MB/s eta 0:00:00
  Created wheel 

#### Imports

In [3]:
%matplotlib inline
import os

os.environ["DGLBACKEND"] = "pytorch"
import dgl
import numpy as np
import networkx as nx
import torch
import torch.nn as nn
import dgl.function as fn
import torch.nn.functional as F
import shutil
from torch.utils.data import DataLoader
import cloudpickle
from dgl.nn import GraphConv
from sklearn.preprocessing import StandardScaler

Now let's access the dataset uploaded in drive:

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!mkdir FreeSolv_dataset

Unzipping dataset into the directory we just made:

In [6]:
!unzip /content/drive/MyDrive/freesolv.zip -d FreeSolv_dataset

Archive:  /content/drive/MyDrive/freesolv.zip
  inflating: FreeSolv_dataset/scaffold_0_smiles_train.pickle  
  inflating: FreeSolv_dataset/scaffold_0_test.bin  
  inflating: FreeSolv_dataset/scaffold_0_val.bin  
  inflating: FreeSolv_dataset/scaffold_0_smiles_val.pickle  
  inflating: FreeSolv_dataset/scaffold_0_smiles_test.pickle  
  inflating: FreeSolv_dataset/scaffold_0_train.bin  


#### Exploring data

In [7]:
train = dgl.load_graphs('FreeSolv_dataset/scaffold_0_train.bin')
type(train)

tuple

In [8]:
train

([Graph(num_nodes=2, num_edges=2,
        ndata_schemes={'v': Scheme(shape=(128,), dtype=torch.float32)}
        edata_schemes={'e': Scheme(shape=(13,), dtype=torch.float32)}),
  Graph(num_nodes=2, num_edges=2,
        ndata_schemes={'v': Scheme(shape=(128,), dtype=torch.float32)}
        edata_schemes={'e': Scheme(shape=(13,), dtype=torch.float32)}),
  Graph(num_nodes=2, num_edges=2,
        ndata_schemes={'v': Scheme(shape=(128,), dtype=torch.float32)}
        edata_schemes={'e': Scheme(shape=(13,), dtype=torch.float32)}),
  Graph(num_nodes=1, num_edges=0,
        ndata_schemes={'v': Scheme(shape=(128,), dtype=torch.float32)}
        edata_schemes={'e': Scheme(shape=(20,), dtype=torch.float32)}),
  Graph(num_nodes=2, num_edges=2,
        ndata_schemes={'v': Scheme(shape=(128,), dtype=torch.float32)}
        edata_schemes={'e': Scheme(shape=(13,), dtype=torch.float32)}),
  Graph(num_nodes=2, num_edges=2,
        ndata_schemes={'v': Scheme(shape=(128,), dtype=torch.float32)}
        ed

The training set is actually a pair: first component is a list of molecular graphs and the second component is a dictionary consist of labels, masks and global features of each graph. The validation and test sets are similar.

In [9]:
train_masks = train[1]['masks']
torch.unique(train_masks)

tensor([1.])

In [10]:
valid = dgl.load_graphs('FreeSolv_dataset/scaffold_0_val.bin')
valid_masks = valid[1]['masks']
torch.unique(valid_masks)

tensor([1.])

In [11]:
test = dgl.load_graphs('FreeSolv_dataset/scaffold_0_test.bin')
test_masks = test[1]['masks']
torch.unique(test_masks)

tensor([1.])

So all values of masks tensor is 1. Thus we do not need masks. So we do not consider it in the following process.

#### Paths

Here we just set some paths for saving the models and parameters;.

In [12]:
current_dir = "./"
checkpoint_path = current_dir + "save_models/model_checkpoints/" + "checkpoint"
os.makedirs(checkpoint_path, exist_ok=True)

best_model_path = current_dir + "save_models/best_model/"

folder_data_temp = current_dir +"data_temp/"
shutil.rmtree(folder_data_temp, ignore_errors=True)

#### Custom PyTorch Dataset

Now we implement a CustomDataset class. We set a scaler and fit it on labels and also we do some reshaping on labels and global feature tensors. This class returns the graph and its coresponding scaled label and global feature for each index.

In [13]:
""" Regression Dataset """
class DGLDatasetReg(torch.utils.data.Dataset):
    def __init__(self, address, train=False, scaler=None):
            self.train = train
            self.scaler = scaler
            self.data_set, train_labels_masks_globals = dgl.load_graphs(address+".bin")
            num_graphs = len(self.data_set)
            self.labels = train_labels_masks_globals["labels"].view(num_graphs,-1)
            self.globals = train_labels_masks_globals["globals"].view(num_graphs,-1)

    def scaler_method(self):
        if self.train:
            scaler = StandardScaler().fit(self.labels)
            self.scaler = scaler
        return self.scaler

    def __len__(self):
        return len(self.data_set)

    def __getitem__(self, idx):
        return  self.data_set[idx], torch.tensor(self.scaler.transform(self.labels)[idx]).float(), self.globals[idx]

#### Defining Train, Validation and Test sets

In [14]:
path_data_temp =  'FreeSolv_dataset/scaffold_0'
train_set = DGLDatasetReg(address=path_data_temp+"_train", train=True)
scaler = train_set.scaler_method()
val_set = DGLDatasetReg(address=path_data_temp+"_val", scaler=scaler)
test_set = DGLDatasetReg(address=path_data_temp+"_test", scaler=scaler)

print('Train set size: ', len(train_set))
print('Validation set size: ', len(val_set))
print('Test set size: ', len(test_set))

Train set size:  513
Validation set size:  64
Test set size:  65


#### DataLoader

Here we load the training, validation and test data with pytorch DataLoader. We also implement a custom collate function as the default one is not going to work on graph data.

In [15]:
def collate(batch):
    # batch is a list of tuples (graphs, labels, masks, globals)
    # Concatenate a sequence of graphs
    graphs = [e[0] for e in batch]
    g = dgl.batch(graphs)

    # Concatenate a sequence of tensors (labels) along a new dimension
    labels = [e[1] for e in batch]
    labels = torch.stack(labels, 0)

    # Concatenate a sequence of tensors (globals) along a new dimension
    globals = [e[2] for e in batch]
    globals = torch.stack(globals, 0)

    return g, labels, globals


def loader(batch_size=64):
    train_dataloader = DataLoader(train_set,
                              batch_size=batch_size,
                              collate_fn=collate,
                              drop_last=True,
                              shuffle=True,
                              num_workers=1)

    val_dataloader =  DataLoader(val_set,
                             batch_size=batch_size,
                             collate_fn=collate,
                             drop_last=True,
                             shuffle=False,
                             num_workers=1)

    test_dataloader = DataLoader(test_set,
                             batch_size=batch_size,
                             collate_fn=collate,
                             drop_last=True,
                             shuffle=False,
                             num_workers=1)
    return train_dataloader, val_dataloader, test_dataloader

In [16]:
train_dataloader, val_dataloader, test_dataloader = loader(batch_size=64)

Let's just observe a batch of train_dataloader:

In [17]:
g, labels, globals = next(iter(train_dataloader))
print(g)
print(globals)

Graph(num_nodes=213, num_edges=310,
      ndata_schemes={'v': Scheme(shape=(128,), dtype=torch.float32)}
      edata_schemes={'e': Scheme(shape=(13,), dtype=torch.float32)})
tensor([[9.7835e-01, 6.1390e-03, 9.1704e-04,  ..., 4.7036e-08, 1.6663e-01,
         4.5457e-01],
        [7.3640e-01, 6.3832e-04, 3.4579e-05,  ..., 4.7036e-08, 1.6663e-01,
         2.9420e-01],
        [9.7644e-01, 4.6711e-03, 9.1704e-04,  ..., 4.7036e-08, 1.6663e-01,
         5.8480e-01],
        ...,
        [9.9173e-01, 1.7805e-03, 1.8314e-03,  ..., 4.7036e-08, 1.6663e-01,
         4.0686e-01],
        [9.6295e-01, 9.2738e-04, 4.5368e-05,  ..., 4.7036e-08, 1.6663e-01,
         2.3199e-01],
        [9.8993e-01, 2.5920e-02, 2.0337e-03,  ..., 4.7036e-08, 1.6663e-01,
         3.3295e-01]])


#### Some variables

In [18]:
# Size of global feature of each graph
global_size = 200

# Number of epochs to train the model
num_epochs = 100

# Number of steps to wait if the model performance on the validation set does not improve
patience = 10

#Configurations to instantiate the model
config = {"node_feature_size":127, "edge_feature_size":12, "hidden_size":100}


#### Function in order to computing score(RMSE)

In [19]:
import math

def compute_score(model, data_loader, val_size):
    model.eval()
    loss_sum = nn.MSELoss(reduction='sum') # MSE with sum instead of mean, i.e., sum_i[(y_i)^2-(y'_i)^2]
    final_loss = 0
    with torch.no_grad():
        for i, (mol_dgl_graph, labels, globals) in enumerate(data_loader):
            prediction = model(mol_dgl_graph, globals)
            prediction = torch.tensor(scaler.inverse_transform(prediction.detach().cpu()))
            labels = torch.tensor(scaler.inverse_transform(labels.cpu()))
            loss = loss_sum(prediction, labels)
            final_loss += loss.item()
        final_loss /= val_size
        final_loss = math.sqrt(final_loss) # RMSE
    return final_loss


#### Loss Function

As the task is regression, we choose MSE loss function.

In [20]:
def loss_func(output, label):
    criterion = nn.MSELoss()
    loss = criterion(output,label)
    return loss

#### Training

Here we implement a function for training the model for one epoch(train_epoch). Then we use it in the main training function(train_evaluate). We save the best model in the path we set before in order to use it in the test phase.

In [21]:
# Training function

def train_epoch(train_dataloader, model, optimizer):
    epoch_train_loss = 0
    iterations = 0
    model.train() # Prepare model for training
    for i, (mol_dgl_graph, labels, globals) in enumerate(train_dataloader):
        prediction = model(mol_dgl_graph, globals)
        loss_train = loss_func(prediction, labels)
        optimizer.zero_grad(set_to_none=True)
        loss_train.backward()
        optimizer.step()
        epoch_train_loss += loss_train.detach().item()
        iterations += 1
    epoch_train_loss /= iterations

    return epoch_train_loss


def train_evaluate(model):
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)

    best_val = np.Inf
    patience_count = 1
    epoch = 1

    while epoch <= num_epochs:
        if patience_count <= patience:
            model.train()
            loss_train = train_epoch(train_dataloader, model, optimizer)
            model.eval()
            score_val = compute_score(model, val_dataloader, len(val_set))
            if score_val < best_val:
                best_val = score_val
                print("Save checkpoint!")
                path = os.path.join(checkpoint_path, 'checkpoint.pth')
                dict_checkpoint = {"score_val": score_val}
                dict_checkpoint.update({"model_state_dict": model.state_dict(), "optimizer_state": optimizer.state_dict()})
                with open(path, "wb") as outputfile:
                    cloudpickle.dump(dict_checkpoint, outputfile)
                patience_count = 1
            else:
                print("Patience", patience_count)
                patience_count += 1

            print("Epoch: {}/{} | Training Loss: {:.3f} | Valid Score: {:.3f}".format(
            epoch, num_epochs, loss_train, score_val))

            print(" ")
            print("Epoch: {}/{} | Best Valid Score Until Now: {:.3f}".format(epoch, num_epochs, best_val), "\n")
        epoch += 1

    # best model save
    shutil.rmtree(best_model_path, ignore_errors=True)
    shutil.copytree(checkpoint_path, best_model_path)

    print("Final results:")
    print("Average Valid Score: {:.3f}".format(np.mean(best_val)), "\n")


#### Function to compute test set score of the final saved model

The following function uses the best model and computes the RMSE score for test set.




In [22]:
import time
start_time = time.time()

def test_evaluate(model):
    path = os.path.join(best_model_path, 'checkpoint.pth')
    with open(path, 'rb') as f:
        checkpoint = cloudpickle.load(f)
    model.load_state_dict(checkpoint["model_state_dict"])
    model.eval()
    test_score = compute_score(model, test_dataloader, len(test_set))

    print("Test Score: {:.3f}".format(test_score), "\n")
    print("Execution time: {:.3f} seconds".format(time.time() - start_time))

# GNNs

#### GCN with two convolutional layers.

Here we use the GraphConv of DGL for message passing.

In [23]:
class GCN1(nn.Module):
    def __init__(self, config, global_size = 200, num_tasks = 1):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks

        # Node feature size
        self.node_feature_size = self.config.get('node_feature_size', 127)

        # Edge feature size
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

        # Hidden size
        self.hidden_size = self.config.get('hidden_size', 100)

        self.conv1 = GraphConv(self.node_feature_size, self.hidden_size, allow_zero_in_degree='True')
        self.conv2 = GraphConv(self.hidden_size, self.num_tasks, allow_zero_in_degree='True')

    # def forward(self, g, in_feat):
    def forward(self, mol_dgl_graph, globals):
        mol_dgl_graph.ndata["v"]= mol_dgl_graph.ndata["v"][:,:self.node_feature_size]
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:,:self.edge_feature_size]

        h = self.conv1(mol_dgl_graph, mol_dgl_graph.ndata["v"])
        h = F.relu(h)
        h = self.conv2(mol_dgl_graph, h)
        mol_dgl_graph.ndata["h"] = h

        return dgl.mean_nodes(mol_dgl_graph, "h")

#### Train the models and evaluate their performance

In [24]:
model_1 = GCN1(config, global_size)

In [25]:
train_evaluate(model_1)
test_evaluate(model_1)

Exception in thread _colab_inspector_thread:
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.10/dist-packages/google/colab/_debugpy.py", line 64, in inspector_thread
    _variable_inspector.run(shell, time)
  File "/usr/local/lib/python3.10/dist-packages/google/colab/_variable_inspector.py", line 27, in run
    globals().clear()
TypeError: 'Tensor' object is not callable


Save checkpoint!
Epoch: 1/100 | Training Loss: 1.019 | Valid Score: 4.540
 
Epoch: 1/100 | Best Valid Score Until Now: 4.540 

Save checkpoint!
Epoch: 2/100 | Training Loss: 1.000 | Valid Score: 4.454
 
Epoch: 2/100 | Best Valid Score Until Now: 4.454 

Save checkpoint!
Epoch: 3/100 | Training Loss: 0.978 | Valid Score: 4.390
 
Epoch: 3/100 | Best Valid Score Until Now: 4.390 

Save checkpoint!
Epoch: 4/100 | Training Loss: 0.965 | Valid Score: 4.330
 
Epoch: 4/100 | Best Valid Score Until Now: 4.330 

Save checkpoint!
Epoch: 5/100 | Training Loss: 0.956 | Valid Score: 4.280
 
Epoch: 5/100 | Best Valid Score Until Now: 4.280 

Save checkpoint!
Epoch: 6/100 | Training Loss: 0.943 | Valid Score: 4.243
 
Epoch: 6/100 | Best Valid Score Until Now: 4.243 

Save checkpoint!
Epoch: 7/100 | Training Loss: 0.930 | Valid Score: 4.203
 
Epoch: 7/100 | Best Valid Score Until Now: 4.203 

Save checkpoint!
Epoch: 8/100 | Training Loss: 0.924 | Valid Score: 4.180
 
Epoch: 8/100 | Best Valid Score Unt

#### GCN with 4 convolutional layers and batch normalization layers and dropouts.

In [26]:
class GCN2(nn.Module):
    def __init__(self, config, global_size = 200, num_tasks = 1):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks

        # Node feature size
        self.node_feature_size = self.config.get('node_feature_size', 127)

        # Edge feature size
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

        # Hidden size
        self.hidden_size = self.config.get('hidden_size', 100)

        self.conv1 = GraphConv(self.node_feature_size, self.hidden_size, allow_zero_in_degree='True')
        self.bn1 = nn.BatchNorm1d(self.hidden_size)
        self.dropout1 = nn.Dropout(0.2)

        self.conv2 = GraphConv(self.hidden_size, self.hidden_size, allow_zero_in_degree='True')
        self.bn2 = nn.BatchNorm1d(self.hidden_size)
        self.dropout2 = nn.Dropout(0.2)

        self.conv3 = GraphConv(self.hidden_size, self.hidden_size, allow_zero_in_degree='True')
        self.bn3 = nn.BatchNorm1d(self.hidden_size)
        self.dropout3 = nn.Dropout(0.2)

        self.conv4 = GraphConv(self.hidden_size, self.num_tasks, allow_zero_in_degree='True')

    # def forward(self, g, in_feat):
    def forward(self, mol_dgl_graph, globals):
        mol_dgl_graph.ndata["v"]= mol_dgl_graph.ndata["v"][:,:self.node_feature_size]
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:,:self.edge_feature_size]

        h = self.conv1(mol_dgl_graph, mol_dgl_graph.ndata["v"])
        h = self.bn1(h)
        h = F.relu(h)
        h = self.dropout1(h)

        h = self.conv2(mol_dgl_graph, h)
        h = self.bn2(h)
        h = F.relu(h)
        h = self.dropout2(h)

        h = self.conv3(mol_dgl_graph, h)
        h = self.bn3(h)
        h = F.relu(h)
        h = self.dropout3(h)

        h = self.conv4(mol_dgl_graph, h)

        mol_dgl_graph.ndata["h"] = h

        return dgl.mean_nodes(mol_dgl_graph, "h")

In [27]:
model_2 = GCN2(config, global_size)

In [45]:
train_evaluate(model_2)
test_evaluate(model_2)

Save checkpoint!
Epoch: 1/100 | Training Loss: 0.641 | Valid Score: 2.986
 
Epoch: 1/100 | Best Valid Score Until Now: 2.986 

Save checkpoint!
Epoch: 2/100 | Training Loss: 0.647 | Valid Score: 2.967
 
Epoch: 2/100 | Best Valid Score Until Now: 2.967 

Save checkpoint!
Epoch: 3/100 | Training Loss: 0.653 | Valid Score: 2.953
 
Epoch: 3/100 | Best Valid Score Until Now: 2.953 

Patience 1
Epoch: 4/100 | Training Loss: 0.637 | Valid Score: 3.012
 
Epoch: 4/100 | Best Valid Score Until Now: 2.953 

Patience 2
Epoch: 5/100 | Training Loss: 0.565 | Valid Score: 3.043
 
Epoch: 5/100 | Best Valid Score Until Now: 2.953 

Patience 3
Epoch: 6/100 | Training Loss: 0.589 | Valid Score: 3.041
 
Epoch: 6/100 | Best Valid Score Until Now: 2.953 

Patience 4
Epoch: 7/100 | Training Loss: 0.560 | Valid Score: 3.032
 
Epoch: 7/100 | Best Valid Score Until Now: 2.953 

Patience 5
Epoch: 8/100 | Training Loss: 0.673 | Valid Score: 3.018
 
Epoch: 8/100 | Best Valid Score Until Now: 2.953 

Patience 6
Epo

Adding more layers and batch normalization and dropouts improved the result.

#### GraphSAGE

Here we implement a SAGEConv for message passing. We only use node features in the following GNN.

In [23]:
class SAGEConv(nn.Module):
    def __init__(self, in_feat, out_feat):
        super(SAGEConv, self).__init__()
        # A linear submodule for projecting the input and neighbor feature to the output.
        self.linear = nn.Linear(in_feat * 2, out_feat)

    def forward(self, g, h):
        with g.local_scope():
            g.ndata["h"] = h
            # update_all is a message passing API.
            g.update_all(
                message_func=fn.copy_u("h", "m"),
                reduce_func=fn.mean("m", "h_N"),
            )
            h_N = g.ndata["h_N"]
            h_total = torch.cat([h, h_N], dim=1)
            return self.linear(h_total)

In [30]:
class GraphSAGE1(nn.Module):
    def __init__(self, config, global_size = 200, num_tasks = 1):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks

        # Node feature size
        self.node_feature_size = self.config.get('node_feature_size', 127)

        # Edge feature size
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

        # Hidden size
        self.hidden_size = self.config.get('hidden_size', 100)

        self.conv1 = SAGEConv(self.node_feature_size, self.hidden_size)
        self.conv2 = SAGEConv(self.hidden_size, self.num_tasks)

    def forward(self, mol_dgl_graph, globals):
        mol_dgl_graph.ndata["v"]= mol_dgl_graph.ndata["v"][:,:self.node_feature_size]
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:,:self.edge_feature_size]
        h = self.conv1(mol_dgl_graph, mol_dgl_graph.ndata["v"])
        h = F.relu(h)
        h = self.conv2(mol_dgl_graph, h)
        mol_dgl_graph.ndata["h"] = h
        return dgl.mean_nodes(mol_dgl_graph, "h")

In [31]:
model_3 = GraphSAGE1(config, global_size)

In [46]:
train_evaluate(model_3)
test_evaluate(model_3)

Save checkpoint!
Epoch: 1/100 | Training Loss: 0.336 | Valid Score: 3.844
 
Epoch: 1/100 | Best Valid Score Until Now: 3.844 

Patience 1
Epoch: 2/100 | Training Loss: 0.332 | Valid Score: 3.844
 
Epoch: 2/100 | Best Valid Score Until Now: 3.844 

Patience 2
Epoch: 3/100 | Training Loss: 0.329 | Valid Score: 3.852
 
Epoch: 3/100 | Best Valid Score Until Now: 3.844 

Patience 3
Epoch: 4/100 | Training Loss: 0.327 | Valid Score: 3.856
 
Epoch: 4/100 | Best Valid Score Until Now: 3.844 

Patience 4
Epoch: 5/100 | Training Loss: 0.325 | Valid Score: 3.854
 
Epoch: 5/100 | Best Valid Score Until Now: 3.844 

Patience 5
Epoch: 6/100 | Training Loss: 0.322 | Valid Score: 3.859
 
Epoch: 6/100 | Best Valid Score Until Now: 3.844 

Patience 6
Epoch: 7/100 | Training Loss: 0.319 | Valid Score: 3.862
 
Epoch: 7/100 | Best Valid Score Until Now: 3.844 

Patience 7
Epoch: 8/100 | Training Loss: 0.319 | Valid Score: 3.867
 
Epoch: 8/100 | Best Valid Score Until Now: 3.844 

Patience 8
Epoch: 9/100 | 

#### GraphSAGE with 4 layers and batch normalization and dropouts.

In [24]:
class GraphSAGE2(nn.Module):
    def __init__(self, config, global_size = 200, num_tasks = 1):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks

        # Node feature size
        self.node_feature_size = self.config.get('node_feature_size', 127)

        # Edge feature size
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

        # Hidden size
        self.hidden_size = self.config.get('hidden_size', 100)

        self.conv1 = SAGEConv(self.node_feature_size, self.hidden_size)
        self.bn1 = nn.BatchNorm1d(self.hidden_size)
        self.dropout1 = nn.Dropout(0.2)

        self.conv2 = SAGEConv(self.hidden_size, self.hidden_size)
        self.bn2 = nn.BatchNorm1d(self.hidden_size)
        self.dropout2 = nn.Dropout(0.2)

        self.conv3 = SAGEConv(self.hidden_size, self.hidden_size)
        self.bn3 = nn.BatchNorm1d(self.hidden_size)
        self.dropout3 = nn.Dropout(0.2)

        self.conv4 = SAGEConv(self.hidden_size, self.num_tasks)

    def forward(self, mol_dgl_graph, globals):
        mol_dgl_graph.ndata["v"]= mol_dgl_graph.ndata["v"][:,:self.node_feature_size]
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:,:self.edge_feature_size]

        h = self.conv1(mol_dgl_graph, mol_dgl_graph.ndata["v"])
        h = self.bn1(h)
        h = F.relu(h)
        h = self.dropout1(h)

        h = self.conv2(mol_dgl_graph, h)
        h = self.bn2(h)
        h = F.relu(h)
        h = self.dropout2(h)

        h = self.conv3(mol_dgl_graph, h)
        h = self.bn3(h)
        h = F.relu(h)
        h = self.dropout3(h)

        h = self.conv4(mol_dgl_graph, h)

        mol_dgl_graph.ndata["h"] = h

        return dgl.mean_nodes(mol_dgl_graph, "h")

In [25]:
model_4 = GraphSAGE2(config, global_size)

In [29]:
train_evaluate(model_4)
test_evaluate(model_4)

Save checkpoint!
Epoch: 1/100 | Training Loss: 0.454 | Valid Score: 3.954
 
Epoch: 1/100 | Best Valid Score Until Now: 3.954 

Save checkpoint!
Epoch: 2/100 | Training Loss: 0.421 | Valid Score: 3.944
 
Epoch: 2/100 | Best Valid Score Until Now: 3.944 

Patience 1
Epoch: 3/100 | Training Loss: 0.399 | Valid Score: 3.977
 
Epoch: 3/100 | Best Valid Score Until Now: 3.944 

Patience 2
Epoch: 4/100 | Training Loss: 0.367 | Valid Score: 3.990
 
Epoch: 4/100 | Best Valid Score Until Now: 3.944 

Patience 3
Epoch: 5/100 | Training Loss: 0.349 | Valid Score: 3.989
 
Epoch: 5/100 | Best Valid Score Until Now: 3.944 

Patience 4
Epoch: 6/100 | Training Loss: 0.359 | Valid Score: 4.025
 
Epoch: 6/100 | Best Valid Score Until Now: 3.944 

Patience 5
Epoch: 7/100 | Training Loss: 0.350 | Valid Score: 4.087
 
Epoch: 7/100 | Best Valid Score Until Now: 3.944 

Patience 6
Epoch: 8/100 | Training Loss: 0.360 | Valid Score: 4.087
 
Epoch: 8/100 | Best Valid Score Until Now: 3.944 

Patience 7
Epoch: 9/

A better result!

#### Custom GNNs

Here we implement a custom GNN. we use u_add_v as message function and mean as aggregation function. We have 4 layers and batch normalization and dropouts in the architecture of the GNN.

In [30]:
class CustomGraphConv1(nn.Module):
    def __init__(self, in_feat, out_feat):
        super(CustomGraphConv1, self).__init__()
        # A linear submodule for projecting the input and neighbor feature to the output.
        self.linear = nn.Linear(in_feat * 2, out_feat)

    def forward(self, g, h):
        with g.local_scope():
            g.ndata["h"] = h
            g.update_all(
                message_func=fn.u_add_v("h", "h", "m"),
                reduce_func=fn.mean("m", "h_N"),
            )
            h_N = g.ndata["h_N"]
            h_total = torch.cat([h, h_N], dim=1)
            return self.linear(h_total)

In [31]:
class GNN(nn.Module):
    def __init__(self, config, global_size = 200, num_tasks = 1):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks

        # Node feature size
        self.node_feature_size = self.config.get('node_feature_size', 127)

        # Edge feature size
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

        # Hidden size
        self.hidden_size = self.config.get('hidden_size', 100)

        self.conv1 = CustomGraphConv1(self.node_feature_size, self.hidden_size)
        self.bn1 = nn.BatchNorm1d(self.hidden_size)
        self.dropout1 = nn.Dropout(0.2)

        self.conv2 = CustomGraphConv1(self.hidden_size, self.hidden_size)
        self.bn2 = nn.BatchNorm1d(self.hidden_size)
        self.dropout2 = nn.Dropout(0.2)

        self.conv3 = CustomGraphConv1(self.hidden_size, self.hidden_size)
        self.bn3 = nn.BatchNorm1d(self.hidden_size)
        self.dropout3 = nn.Dropout(0.2)

        self.conv4 = CustomGraphConv1(self.hidden_size, self.num_tasks)

    def forward(self, mol_dgl_graph, globals):
        mol_dgl_graph.ndata["v"]= mol_dgl_graph.ndata["v"][:,:self.node_feature_size]
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:,:self.edge_feature_size]

        h = self.conv1(mol_dgl_graph, mol_dgl_graph.ndata["v"])
        h = self.bn1(h)
        h = F.relu(h)
        h = self.dropout1(h)

        h = self.conv2(mol_dgl_graph, h)
        h = self.bn2(h)
        h = F.relu(h)
        h = self.dropout2(h)

        h = self.conv3(mol_dgl_graph, h)
        h = self.bn3(h)
        h = F.relu(h)
        h = self.dropout3(h)

        h = self.conv4(mol_dgl_graph, h)

        mol_dgl_graph.ndata["h"] = h

        return dgl.mean_nodes(mol_dgl_graph, "h")

In [33]:
model_5 = GNN(config, global_size)

In [35]:
train_evaluate(model_5)
test_evaluate(model_5)

Save checkpoint!
Epoch: 1/100 | Training Loss: 0.452 | Valid Score: 3.671
 
Epoch: 1/100 | Best Valid Score Until Now: 3.671 

Patience 1
Epoch: 2/100 | Training Loss: 0.480 | Valid Score: 3.711
 
Epoch: 2/100 | Best Valid Score Until Now: 3.671 

Patience 2
Epoch: 3/100 | Training Loss: 0.457 | Valid Score: 3.779
 
Epoch: 3/100 | Best Valid Score Until Now: 3.671 

Patience 3
Epoch: 4/100 | Training Loss: 0.425 | Valid Score: 3.859
 
Epoch: 4/100 | Best Valid Score Until Now: 3.671 

Patience 4
Epoch: 5/100 | Training Loss: 0.407 | Valid Score: 3.945
 
Epoch: 5/100 | Best Valid Score Until Now: 3.671 

Patience 5
Epoch: 6/100 | Training Loss: 0.423 | Valid Score: 3.970
 
Epoch: 6/100 | Best Valid Score Until Now: 3.671 

Patience 6
Epoch: 7/100 | Training Loss: 0.399 | Valid Score: 3.878
 
Epoch: 7/100 | Best Valid Score Until Now: 3.671 

Patience 7
Epoch: 8/100 | Training Loss: 0.358 | Valid Score: 3.820
 
Epoch: 8/100 | Best Valid Score Until Now: 3.671 

Patience 8
Epoch: 9/100 | 

Using edge features for message passing:

In [23]:
class CustomGraphConv2(nn.Module):
    def __init__(self, in_feat, out_feat):
        super(CustomGraphConv2, self).__init__()
        self.linear = nn.Linear(in_feat + 12 , out_feat)

    def forward(self, g, h, w):
        with g.local_scope():
            g.ndata["h"] = h
            g.edata["w"] = w
            g.update_all(
                message_func=fn.copy_e("w", "m"), # computing message using edge feature
                reduce_func=fn.sum("m", "h_N"),
            )
            h_N = g.ndata["h_N"]
            h_total = torch.cat([h, h_N], dim=1)
            # print(h_total.shape)
            return self.linear(h_total)

In [24]:
class GNN2(nn.Module):
    def __init__(self, config, global_size = 200, num_tasks = 1):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks

        # Node feature size
        self.node_feature_size = self.config.get('node_feature_size', 127)

        # Edge feature size
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

        # Hidden size
        self.hidden_size = self.config.get('hidden_size', 100)

        self.conv1 = CustomGraphConv2(self.node_feature_size, self.hidden_size)
        self.bn1 = nn.BatchNorm1d(self.hidden_size)
        self.dropout1 = nn.Dropout(0.2)

        self.conv2 = CustomGraphConv2(self.hidden_size, self.hidden_size)
        self.bn2 = nn.BatchNorm1d(self.hidden_size)
        self.dropout2 = nn.Dropout(0.2)

        self.conv3 = CustomGraphConv2(self.hidden_size, self.hidden_size)
        self.bn3 = nn.BatchNorm1d(self.hidden_size)
        self.dropout3 = nn.Dropout(0.2)

        self.conv4 = CustomGraphConv2(self.hidden_size, self.num_tasks)

    def forward(self, mol_dgl_graph, globals):
        mol_dgl_graph.ndata["v"]= mol_dgl_graph.ndata["v"][:,:self.node_feature_size]
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:,:self.edge_feature_size]

        h = self.conv1(mol_dgl_graph, mol_dgl_graph.ndata["v"], mol_dgl_graph.edata["e"])
        h = self.bn1(h)
        h = F.relu(h)
        h = self.dropout1(h)

        h = self.conv2(mol_dgl_graph, h, mol_dgl_graph.edata["e"])
        h = self.bn2(h)
        h = F.relu(h)
        h = self.dropout2(h)

        h = self.conv3(mol_dgl_graph, h, mol_dgl_graph.edata["e"])
        h = self.bn3(h)
        h = F.relu(h)
        h = self.dropout3(h)

        h = self.conv4(mol_dgl_graph, h, mol_dgl_graph.edata["e"])

        mol_dgl_graph.ndata["h"] = h

        return dgl.mean_nodes(mol_dgl_graph, "h")

In [25]:
model_6 = GNN2(config, global_size)

In [27]:
train_evaluate(model_6)
test_evaluate(model_6)

Save checkpoint!
Epoch: 1/100 | Training Loss: 0.452 | Valid Score: 3.712
 
Epoch: 1/100 | Best Valid Score Until Now: 3.712 

Patience 1
Epoch: 2/100 | Training Loss: 0.425 | Valid Score: 3.728
 
Epoch: 2/100 | Best Valid Score Until Now: 3.712 

Patience 2
Epoch: 3/100 | Training Loss: 0.420 | Valid Score: 3.724
 
Epoch: 3/100 | Best Valid Score Until Now: 3.712 

Patience 3
Epoch: 4/100 | Training Loss: 0.404 | Valid Score: 3.741
 
Epoch: 4/100 | Best Valid Score Until Now: 3.712 

Patience 4
Epoch: 5/100 | Training Loss: 0.405 | Valid Score: 3.713
 
Epoch: 5/100 | Best Valid Score Until Now: 3.712 

Patience 5
Epoch: 6/100 | Training Loss: 0.351 | Valid Score: 3.728
 
Epoch: 6/100 | Best Valid Score Until Now: 3.712 

Patience 6
Epoch: 7/100 | Training Loss: 0.392 | Valid Score: 3.742
 
Epoch: 7/100 | Best Valid Score Until Now: 3.712 

Patience 7
Epoch: 8/100 | Training Loss: 0.393 | Valid Score: 3.729
 
Epoch: 8/100 | Best Valid Score Until Now: 3.712 

Patience 8
Epoch: 9/100 | 